# Nicholas Tam Analysis

## Research Question/Interests
Research question: Given the tree dataset, how would we want to arrange the types of trees to be planted in certain locations, such that they could survive and significantly improve the ecosystem?

Given the research question, it would be ideal to analyze relevant properties, such as the relative sizes of each location and the proportions of planted trees for each species.

The following questions will be used as guidelines to supplement the question
- Which planted species exist in many different locations within each neighbourhood?
- How do trees that have been planted directly compare in terms of diameter to those that have grown naturally?
- What is the distribution of ages for specific tree species?

## Task Analysis
- Compute: Compute the average percentage differences in `DIAMETER` between planted and non-planted trees, and the ages by the number of days between `DATE_PLANTED` and March 5 2025.
- Filter: Filter out `SPECIES_NAME` such that there are plenty of both planted and non-planted trees.
- Characterise distribution: Find the distribution of ages for planted tree `SPECIES_NAME`?
- Sort: Rank `SPECIES_NAME` for appearing in the greatest number of `NEIGHBOURHOOD_NAME` and having large proportions of each corresponding`ON_STREET` containing them.
- Retrieve value: Retrieve the number of trees that have been planted, separated by `SPECIES_NAME`. 

In [23]:
import altair as alt
import pandas as pd
from toolz.curried import pipe
import os
from pygbif import species

# Create a new data transformer that stores the files in a directory
def json_dir(data, data_dir='altairdata'):
    os.makedirs(data_dir, exist_ok=True)
    return pipe(data, alt.to_json(filename=data_dir + '/{prefix}-{hash}.{extension}') )

# Register and enable the new transformer
alt.data_transformers.register('json_dir', json_dir)
alt.data_transformers.enable('json_dir')

# Handle large data sets (default shows only 5000)
# See here: https://altair-viz.github.io/user_guide/data_transformers.html
alt.data_transformers.disable_max_rows()

alt.renderers.enable('jupyterlab')

RendererRegistry.enable('jupyterlab')

## Data Cleaning
### Initial Dataset
- `TREE_ID`, `CIVIC_NUMBER`, `ON_STREET_BLOCK`, and `HEIGHT_RANGE_ID` are all treated as integers, `DIAMETER` is treated as a float, and the rest are treated as objects.
- `CULTIVAR_NAME`, `NEIGHBOURHOOD_NAME`, and `DATE_PLANTED` are columns that have missing values.
- `public_trees.nunique()` displays the number of unique values for each column. Notably, there is a discrepancy between the number of unique values for `HEIGHT_RANGE_ID` and the number of unique values for `HEIGHT_RANGE`. Since `GENUS_NAME` and `SPECIES_NAME`, along with `ON_STREET` and `ON_STREET_BLOCK`, are related to each other, the number of unique values may be more accurate if they are combined.
- `public_trees.describe()` is used to further display that `TREE_ID`, `CIVIC_NUMBER`, `ON_STREET_BLOCK`, `HEIGHT_RANGE_ID` and `DIAMETER` are all treated as numeric, when only `HEIGHT_RANGE_ID` and `DIAMETER` could be numeric, along with the latitude and longitude from `geo_point_2d`.

In [48]:
import sys
sys.path.append("../../code")
import data_clean

public_trees = pd.read_csv("../../data/raw/public-trees.csv", delimiter=";")

In [ ]:
display(public_trees.info())

In [ ]:
cols_with_nan = public_trees.columns[public_trees.isna().any()].tolist()
display(cols_with_nan)

In [17]:
display(public_trees.nunique())

TREE_ID               181501
CIVIC_NUMBER            8339
STD_STREET               814
GENUS_NAME               169
SPECIES_NAME             539
CULTIVAR_NAME            607
COMMON_NAME             1297
ON_STREET_BLOCK          189
ON_STREET                822
NEIGHBOURHOOD_NAME        22
STREET_SIDE_NAME           6
HEIGHT_RANGE_ID           11
HEIGHT_RANGE               9
DIAMETER                 490
DATE_PLANTED            4570
Geom                  181372
geo_point_2d          181372
dtype: int64

### Cleaning
- `HEIGHT_RANGE` is set to be an ordinal attribute, with the order provided in `priority_order`.
- `TREE_ID`, `CIVIC_NUMBER`, and `ON_STREET_BLOCK` are set to be categorical attributes.
- Earlier EDA provided `CULTIVAR_NAME`, `NEIGHBOURHOOD_NAME`, `DATE_PLANTED` as having NA values; `CULTIVAR_NAME` is filled with the corresponding `SPECIES_NAME`, `NEIGHBOURHOOD_NAME` has `NA` strings applied instead, and `DATE_PLANTED` is left with `None`.
- The latitude and longitude values were initially joined together in a single coordinate column `geo_point_2d`, and thus are split into new columns `LATITUDE` and `LONGITUDE`.
- The values `9`, `0` and `10` all correspond to `HEIGHT_RANGE == '>90'` and thus `0` and `10` are replaced with value `9`.
- `NOMENCLATURE` is a column derived from combining `GENUS_NAME` and `SPECIES_NAME` into 1 string.
- `ON_ADDRESS` is a column derived from combining `ON_STREET` and `ON_STREET_BLOCK` into 1 string.
- `FAMILY_NAME` is a column derived from using the functions in `code/preprocess_query_family.py`, which uses the `GENUS_NAME` column and `from pygbif import species` to provide the corresponding family of plants.

In [50]:
public_trees_cleaned = data_clean.clean_public_trees(public_trees)

FAMILY_NAME already exists. Overwriting the column.
FAMILY_NAME added/overwritten in CSV.
HAS_POLLEN already exists. Replacing the column.
HAS_POLLEN replaced in the CSV.


### Processed Data
- Upon loading `public_trees_cleaned.csv` in the future, need to set the attribute types for `HEIGHT_RANGE`, `TREE_ID`, `CIVIC_NUMBER`, and `ON_STREET_BLOCK`.

In [ ]:
public_trees_cleaned = pd.read_csv("../../data/processed/public_trees_cleaned.csv")
priority_order = ['10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90', '>90']
public_trees_cleaned['HEIGHT_RANGE'] = pd.Categorical(public_trees_cleaned['HEIGHT_RANGE'], categories=priority_order, ordered=True)
public_trees_cleaned[['TREE_ID', 'CIVIC_NUMBER', 'ON_STREET_BLOCK']] = public_trees_cleaned[['TREE_ID', 'CIVIC_NUMBER', 'ON_STREET_BLOCK']].astype(str)

public_trees_cleaned.head()

In [ ]:
display(public_trees_cleaned.info())

In [ ]:
display(public_trees_cleaned.nunique())

In [ ]:
display(public_trees_cleaned.describe())

## EDA 
### 1. Count of each `SPECIES_NAME` that have been planted
- `Planted_Count` for each species is derived by counting the number of rows with `DATE_PLANTED` filled for each tree species.
- `Proportion_Planted` derived from dividing `Planted_Count` by the total number of the given species.
- Would ideally want a reasonably large `Total_Count` value (at least 300) to mitigate the risk of having too few samples of each species to effectively measure and compare characteristics.
- Would also want `Proportion_Planted` to be within certain range of values (between 0.33 and 0.67) to mitigate the possibility that the obtained properties of planted trees are by random chance, and ensure that there are plenty of both planted and non-planted trees to compare against each other.
- Applied log-transform to `Total_Count` to make colored value range easier to interpret.
- Greater values of `Total_Count` generally provide less extreme values of `Proportion_Planted`.

In [ ]:
species_planted_count = public_trees_cleaned[public_trees_cleaned['DATE_PLANTED'].notnull()].groupby('SPECIES_NAME').size().reset_index(name='Planted_Count')
species_total_count = public_trees_cleaned.groupby('SPECIES_NAME').size().reset_index(name='Total_Count')
species_summary = species_planted_count.merge(species_total_count, on='SPECIES_NAME')
species_summary['Proportion_Planted'] = species_summary['Planted_Count'] / species_summary['Total_Count']
# species_summary[(species_summary['Total_Count'] >= 300) & (species_summary['Proportion_Planted'] >= 0.33) & (species_summary['Proportion_Planted'] <= 0.67)]

In [ ]:
species_chart = alt.Chart(species_summary).mark_bar().encode(
    alt.X('SPECIES_NAME:N', sort='color'),
    alt.Y('Proportion_Planted:Q'),
    alt.Color('Total_Count', scale=alt.Scale(type='log'), title='log_{10}(Total_Count)'),
    alt.Tooltip(['Total_Count']),
).properties(
    title='Proportions of planted trees for species with date data',
)

horizontal_lines = alt.Chart(pd.DataFrame({
    'Proportion_Planted': [0.33, 0.67],
})).mark_rule(color='red', strokeDash=[5, 5]).encode(
    alt.Y('Proportion_Planted:Q'),
)

species_chart + horizontal_lines

### 2. Scatterplot of `LATITUDE` against `LONGITUDE`
- Distribution of data points along axes of latitude and longitude.
- Provides an estimated "map" of the areas for each `NEIGHBOURHOOD_NAME` in Vancouver, and thus a better understanding of how tree density could be influenced for each area.
- Limited by the number of `NEIGHBOURHOOD_NAME` exceeding the number of colors
- Could be improved upon with `mark_geoshape()`, may require getting coordinate values that were removed in cleaning.

In [ ]:
latvlong = alt.Chart(public_trees_cleaned).mark_circle().encode(
    alt.X('LATITUDE:Q', scale=alt.Scale(zero=False)),
    alt.Y('LONGITUDE:Q', scale=alt.Scale(zero=False)),
    alt.Color('NEIGHBOURHOOD_NAME:N'),
    alt.Tooltip(['NEIGHBOURHOOD_NAME']),
).properties(
    title='Scatterplot of Latitude and Longitude',
    width = 450, 
    height = 450
)
latvlong

### 3. Boxplots of `DIAMETER` grouped by `HEIGHT_RANGE`
- Box plots of `DIAMETER` against `HEIGHT_RANGE`.
- For the most part, mean `DIAMETER` increases with `HEIGHT_RANGE`.
- Outliers for `DIAMETER` appear to be more frequent for lower `HEIGHT_RANGE` values, and less frequent for higher `HEIGHT_RANGE` values.

In [ ]:
# public_trees_summary = public_trees_cleaned.groupby("HEIGHT_RANGE")["DIAMETER"].describe()
# upper_bound = max(public_trees_summary["75%"] + public_trees_summary["std"] * 3)
# filtered_data = public_trees_cleaned[public_trees_cleaned["DIAMETER"] <= upper_bound]

In [ ]:
public_trees_box = alt.Chart(public_trees_cleaned).mark_boxplot(
    median={"color": "red"},
    extent=3,
    ticks= True,
).encode(
    # alt.X('DIAMETER:Q', scale=alt.Scale(zero=False, domain=[0, upper_bound + 10])),
    alt.X('DIAMETER:Q', scale=alt.Scale(zero=False)),
    alt.Y('HEIGHT_RANGE:N'),
    alt.Color('HEIGHT_RANGE:N'),
    # alt.Column('HEIGHT_RANGE:N'),
).properties(
    title='Boxplots of diameter by height ranges',
    width = 900, 
    height = 200
)
public_trees_box

### 4. Number of `ON_STREET` grouped by `NEIGHBOURHOOD_NAME`
- Number of streets for each neighbourhood.
- Provides idea of potential variety in tree numbers for each location.

In [ ]:
species_per_genus = public_trees_cleaned.groupby('NEIGHBOURHOOD_NAME')['ON_STREET'].nunique().reset_index()

species_per_genus_chart = alt.Chart(species_per_genus).mark_bar().encode(
    alt.X('NEIGHBOURHOOD_NAME:N', title='Neighbourhood'),
    alt.Y('ON_STREET:Q', title='Number of Streets'),
    alt.Tooltip(['NEIGHBOURHOOD_NAME', 'ON_STREET']),
).properties(
    title='Number of streets per neighbourhood',
)
species_per_genus_chart

### 5. Proportions of `HEIGHT_RANGE` for each `NEIGHBOURHOOD_NAME`
- Normalized stacked bar plot for `HEIGHT_RANGE` proportions against `NEIGHBOURHOOD_NAME`.
- For the most part, the majority of `HEIGHT_RANGE` values are between `HEIGHT_RANGE == "10-20"` and `HEIGHT_RANGE == "30-40"`.

In [ ]:
# Aggregate count of trees per neighbourhood and height range
height_counts = public_trees_cleaned.groupby(['NEIGHBOURHOOD_NAME', 'HEIGHT_RANGE']).size().reset_index(name='Count')

# Create the stacked bar chart
neighbour_height_stack = alt.Chart(height_counts).mark_bar().encode(
    alt.Y('NEIGHBOURHOOD_NAME:N', title='Neighbourhood', sort='-y'),  # Sort by count
    alt.X('Count:Q', title='Proportion of Trees Heights', stack='normalize'),
    alt.Color('HEIGHT_RANGE:N', title='Height Range'),  # Custom colors
    alt.Tooltip(['NEIGHBOURHOOD_NAME', 'HEIGHT_RANGE', 'Count']),
).properties(
    title='Stacked Bar Plot of Tree Height Ranges by Neighbourhood',
    width=1000
)

neighbour_height_stack

## Preliminary Sketches

The following sketches assume that `SPECIES_NAME` is filtered such that they follow the rules requested in EDA (1): 
- A reasonably large `Total_Count` value (at least 300) to mitigate the risk of having too few samples of each species to effectively measure and compare characteristics.
- `Proportion_Planted` to be within certain range of values (between 0.33 and 0.67) to mitigate the possibility that the obtained properties of planted trees are by random chance, and ensure that there are plenty of both planted and non-planted trees to compare against each other.

Sketch 1: Which planted species exist in many different locations within each neighbourhood?
- The sketch is a heatmap, with `NEIGHBOURHOOD_NAME` on the x-axis, `SPECIES_NAME` on the y-axis, and colour indicating the proportion of streets within each `NEIGHBOURHOOD_NAME` with at least one instance of `SPECIES_NAME`. This sketch provides an understanding of which species are more common in larger varieties of environments and which species are rarer in comparison.

Sketch 2: How do trees that have been planted directly compare in terms of diameter to those that have grown naturally?
- The sketch is a bar plot, with `SPECIES_NAME` on the x-axis, and the percentage difference in `DIAMETER` between planted and non-planted `SPECIES_NAME` trees on the y-axis, with non-planted trees as the baseline. This sketch provides information on how well planted trees fare in terms of growth  compared to non-planted trees, and how effective they are in their current environments.

Sketch 3: What is the distribution of ages for specific tree species?
- The sketch is a density plot, with age (calculated by the number of days between the planting date and March 5 2025) on the x-axis, and the density on the y-axis, faceted by `SPECIES_NAME`. This sketch helps with identifying which tree species have longer-standing populations and which ones are relatively newer.

![Sketch 3](../../images/sketch_3.jpg)

## High-fidelity Sketch

The high-fidelity sketch selected was a dot plot version of the sketch for the question "How do `SPECIES_NAME` trees that have been planted directly compare in terms of `DIAMETER` to those that have grown naturally?". The sketch adheres to the principles of effectiveness, as the magnitude channel used is position on an aligned scale, allowing for greater accuracy and discriminability. It also adheres to the Gestalt principle of continuity by being arranged in descending order of `DIAM_DIFF_PERCENT`, the percentage difference in diameter between the planted trees and the non-planted trees.

In [ ]:
species_planted_count = public_trees_cleaned[public_trees_cleaned['DATE_PLANTED'].notnull()].groupby('SPECIES_NAME').size().reset_index(name='Planted_Count')
species_total_count = public_trees_cleaned.groupby('SPECIES_NAME').size().reset_index(name='Total_Count')
species_summary = species_planted_count.merge(species_total_count, on='SPECIES_NAME')
species_summary['Proportion_Planted'] = species_summary['Planted_Count'] / species_summary['Total_Count']
planted_species = species_summary[
    (species_summary['Total_Count'] >= 300) & 
    (species_summary['Proportion_Planted'] >= 0.33) & 
    (species_summary['Proportion_Planted'] <= 0.67)
]["SPECIES_NAME"].tolist()

In [ ]:
planted_species_dt = public_trees_cleaned[public_trees_cleaned['SPECIES_NAME'].isin(planted_species)]
planted_species_data = planted_species_dt.copy()
planted_species_data.loc[:, 'PLANTED_VAN'] = planted_species_data['DATE_PLANTED'].notnull()
avg_diameter_by_species = planted_species_data.groupby(['SPECIES_NAME', 'PLANTED_VAN'])['DIAMETER'].mean().unstack().rename(columns={True: 'Planted', False: 'Non-Planted'}).reset_index()

avg_diameter_by_species['DIAM_DIFF_PERCENT'] = (avg_diameter_by_species['Planted'] - avg_diameter_by_species['Non-Planted'])/avg_diameter_by_species['Non-Planted'] * 100
# display(avg_diameter_by_species)
hi_fidelity_sketch = alt.Chart(avg_diameter_by_species).mark_circle(opacity=1).encode(
    alt.X('SPECIES_NAME:N', sort='-y'), 
    alt.Y('DIAM_DIFF_PERCENT:Q', scale=alt.Scale(zero=False)),
    # alt.Color('SPECIES_NAME:N', legend=None),
)
hi_fidelity_sketch